In [3]:
import pandas as pd
import os
import json
import cv2

In [4]:
LABELS = ['AUDIO', 'AUDIO_INPUT', 'AUDIO_OUTPUT', 'DISPLAYPORT', 'DVI', 'ETHERNET', 'HDMI', 'PARALLEL', 'POWER', 'PS', 'PS2', 'USB', 'USBC', 'VGA']

# Load data
df = pd.read_csv('./content/labeled_data.csv')

In [5]:
# Fix img paths
def fix_path(img_path):
    base_name = os.path.basename(img_path)
    return base_name.split('-')[-1]

df['image'] = df['image'].apply(fix_path)

In [8]:

# Function to draw bounding boxes and labels on image
def draw_boxes_and_labels(row, input_path, output_path):
    # Load the image
    img = cv2.imread(os.path.join(input_path, row['image']))

    labels = json.loads(row['label'])
    
    # Iterate over each bounding box and label
    for label in labels:
        # Calculate actual dimensions from percentages
        x_top = int(label['x'] / 100.0 * 800)
        y_top = int(label['y'] / 100.0 * 800)

        width = int(label['width'] / 100.0 * 800)
        height = int(label['height'] / 100.0 * 800)

        x1 = x_top
        y1 = y_top
        x2 = x_top + width
        y2 = y_top + height
        
        # Draw the bounding box
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green color for bounding box

        # Draw the label text
        label_name = label['rectanglelabels'][0]
        cv2.putText(img, label_name, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)  # Blue color for text
    
    # Save the processed image
    cv2.imwrite(os.path.join(output_path, row['image']), img)

# Define your input and output directories
input_image_dir = './content/img_crop' 
output_dir = './content/output'

# Ensure output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each image in the DataFrame
df.apply(lambda row: draw_boxes_and_labels(row, input_image_dir, output_dir), axis=1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
38    None
39    None
40    None
41    None
42    None
43    None
44    None
45    None
46    None
47    None
48    None
49    None
50    None
51    None
52    None
53    None
54    None
dtype: object